<h1><center>Joining data</center></h1>
<h3><center>Joining data from Olist database for analysis</center></h3>

Olist is a Brazilian E-Commerce giant. It sells products of all categories from furniture to electronics. Olist published their database of customers and orders placed between 2016 and 2018. The database consists of multiple files giving cutomer locations, seller locations, orders, payments and shipping details - <a href="https://www.kaggle.com/olistbr/brazilian-ecommerce?select=olist_orders_dataset.csv">Link</a><br>

These files containing various data has to be joined to perform meaningful analysis on them. This notebook containts code for combining the files and get the data ready for descriptive and predictive analysis.

In [34]:
import pandas as pd
import numpy as np
from googletrans import Translator

### Customer data

The olist_customers_dataset contains location informations of the customer. customer_unique_id is a unique id identifying customers in the system. It is an id generated at the time of Signup. The customer_id is a temporary id generated everytime the customer places an order.

In [35]:
cust = pd.read_csv('/home/raj/Downloads/Olist-Ecomm data/olist_customers_dataset.csv')
cust.head(5)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


The olist_geolocation_dataset contains latitude and longitude points, city and state for the zipcodes. Every customer in the customer dataset has a zipcode associated as their location within the state and city.

In [36]:
geoloc = pd.read_csv('/home/raj/Downloads/Olist-Ecomm data/olist_geolocation_dataset.csv')
geoloc.head(5)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


The customer dataset and geolocation dataset are joined on zipcodes. Since one zipcode has multiple locations, the data is grouped on zipcode, and a centroid of all latitude and longitude points for that zipcode is reported. This way customers from one zipcode are mapped to one latitude and longitude point.

In [37]:
loc_coordnts = geoloc[[
    'geolocation_zip_code_prefix',
    'geolocation_lat',
    'geolocation_lng'
]].groupby('geolocation_zip_code_prefix').mean().reset_index()

cols= [
    'customer_id',
    'customer_unique_id',
    'customer_zip_code_prefix',
    'customer_city',
    'customer_state',
    'geolocation_lat',
    'geolocation_lng'
]

customer_data = pd.merge(
    left= cust,
    right= loc_coordnts,
    left_on= 'customer_zip_code_prefix',
    right_on= 'geolocation_zip_code_prefix',
    how= 'left',
)[cols]

customer_data.head(5)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_lat,geolocation_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,-20.498489,-47.396929
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,-23.727992,-46.542848
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.531642,-46.656289
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,-23.499702,-46.185233
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,-22.975100,-47.142925


This combined data gives the location (latitude and longitude) of the customer at the time of order placement narrowed down to the centroid of zipcode.

In [188]:
customer_data.to_csv('./Olist_data/customer_data.csv')

### Order data

The olist_orders_dataset contains mapping of temporary customer_id and the order_id. A new customer_id is generated everytime an order is placed at the same time, a new order_id is generated. Apart from the mapping of customer to the order, the dataset contains timestamps for various milestones during delivery of the product such as purchase date, approval date, shipping date, local carrier date, delivery date and finally as estimated delivery date.

In [38]:
orders = pd.read_csv('/home/raj/Downloads/Olist-Ecomm data/olist_orders_dataset.csv')
orders.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


The olist_order_items_dataset contains a mapping of the order_id and the products bought via product_id. For multiple items bought together or same item bought in quantity more than 1, order_id is repeated for that many rows with a sequence number order_item_id becoming the primary key. The dataset contains price of each product and freight value for it. The seller who has listed the product and the deadline for shipment is also recorded individually for each product.

In [39]:
ord_items = pd.read_csv('/home/raj/Downloads/Olist-Ecomm data/olist_order_items_dataset.csv')
ord_items.head(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


The two datasets above are combined on order_id to get a mapping of customer to the products purchased and the price of the products. The timestamp of the purchase date and freight value are also taken. This new combined dataset gives information on customer's orders - what was purchased? How much was it for? and when was it purchased?

In [40]:
cols= [
    'order_id',
    'customer_id',
    'order_purchase_timestamp',
    'order_item_id',
    'product_id',
    'price',
    'freight_value'
]

complete_order = pd.merge(
    left= orders,
    right= ord_items,
    on= 'order_id',
    how= 'inner'
)[cols]

complete_order.head(5)

,order_id,customer_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,1,87285b34884572647811a353c7ac498a,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,1,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,1,aa4383b373c6aca5d8797843e5594415,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,1,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,1,65266b2da20d04dbe00c5c2d3bb7859e,19.90,8.72


The above dataset is combined with the customer dataset to get a mapping of the customer_unique_id and the orders joined on customer_id. This gives a clear mapping of every customer and the orders the customers have placed over the duration of 2 years (2016 - 1028).

In [41]:
cols = [
    'order_id',
    'order_purchase_timestamp',
    'order_item_id',
    'product_id',
    'price',
    'freight_value',
    'customer_unique_id'
]

cust_orders = pd.merge(
    left= complete_order,
    right= cust,
    on= 'customer_id',
    how= 'inner'
)[cols]

cust_orders.head(5)

,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,customer_unique_id
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,1,87285b34884572647811a353c7ac498a,29.99,8.72,7c396fd4830fd04220f754e42b4e5bff
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,1,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76,af07308b275d755c9edb36a90c618231
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,1,aa4383b373c6aca5d8797843e5594415,159.90,19.22,3a653a41f6f9fc3d2a113cf8398680e8
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,1,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,27.20,7c142cf63193a1473d2e66489a9ae977
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,1,65266b2da20d04dbe00c5c2d3bb7859e,19.90,8.72,72632f0f9dd73dfee390c9b22eb56dd6


In [203]:
cust_orders.to_csv('./Olist_data/customer_order.csv', index= False)

### Product category data

For every product listed on the website, a unique product_id is given. The database doesnot contain the name of the products rather just the category of the product is given along with dimensions of the product and other descriptive statistics. 

In [42]:
products = pd.read_csv('/home/raj/Downloads/Olist-Ecomm data/olist_products_dataset.csv')
products.head(5)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


A separate file gives the translation of the product category names from Portuguese to English.

In [43]:
transln = pd.read_csv('/home/raj/Downloads/Olist-Ecomm data/product_category_name_translation.csv')
transln.head(5)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


The two datasets above our combined to get the product category names in English.

In [44]:
products = pd.merge(
    left= products,
    right= transln,
    on= 'product_category_name',
    how= 'left'
)

products.head(5)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares


This product information has to be now added to the customer order mapping created before this. The previously joined and created dataset which contains informations on orders of every customer is read in.

In [45]:
cust_order = pd.read_csv('/home/raj/Github/Olist-business-analysis/Joined data/customer_order.csv')
cust_order.head(5)

,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,customer_unique_id,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,1,87285b34884572647811a353c7ac498a,29.99,8.72,7c396fd4830fd04220f754e42b4e5bff,housewares
1,128e10d95713541c87cd1a2e48201934,2017-08-15 18:29:31,1,87285b34884572647811a353c7ac498a,29.99,7.78,3a51803cc0d012c3b5dc8b7528cb05f7,housewares
2,0e7e841ddf8f8f2de2bad69267ecfbcf,2017-08-02 18:24:47,1,87285b34884572647811a353c7ac498a,29.99,7.78,ef0996a1a279c26e7ecbd737be23d235,housewares
3,bfc39df4f36c3693ff3b63fcbea9e90a,2017-10-23 23:26:46,1,87285b34884572647811a353c7ac498a,29.99,14.10,e781fdcc107d13d865fc7698711cc572,housewares
4,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,1,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76,af07308b275d755c9edb36a90c618231,perfumery


The customer orders dataset is combined with the products dataset to get information on the category of purchase/order by the customer. Now the new dataset contains information of every order in terms of product, product category, price and shipping costs of the product and the finally the timestamp of purchase.

In [31]:
cust_order = pd.merge(
    left= cust_order,
    right= products[['product_id','product_category_name_english']],
    on= 'product_id',
    how= 'inner'
)

cust_order.head(5)

,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,customer_unique_id,product_category_name_english_x,product_category_name_english_y
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,1,87285b34884572647811a353c7ac498a,29.99,8.72,7c396fd4830fd04220f754e42b4e5bff,housewares,housewares
1,128e10d95713541c87cd1a2e48201934,2017-08-15 18:29:31,1,87285b34884572647811a353c7ac498a,29.99,7.78,3a51803cc0d012c3b5dc8b7528cb05f7,housewares,housewares
2,0e7e841ddf8f8f2de2bad69267ecfbcf,2017-08-02 18:24:47,1,87285b34884572647811a353c7ac498a,29.99,7.78,ef0996a1a279c26e7ecbd737be23d235,housewares,housewares
3,bfc39df4f36c3693ff3b63fcbea9e90a,2017-10-23 23:26:46,1,87285b34884572647811a353c7ac498a,29.99,14.10,e781fdcc107d13d865fc7698711cc572,housewares,housewares
4,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,1,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76,af07308b275d755c9edb36a90c618231,perfumery,perfumery


In [209]:
cust_order.to_csv('./Olist_data/customer_order.csv', index= False)

### Transaction data

For the analysis of the products and to find associations between the various products and product categories sold on the website, a transaction dataset has to be created. For this, the existing dataset of all customers and their orders is manipulated.<br>

The orders are grouped by customer_unique_id and timestamp giving an idea of which customer bought which products and when. The product category is used to create dummy columns which are summed together based on groups. The resultant dataset now gives inforamtion on a customer's order - the time and the quantity of products bought in each order.

In [46]:
txn_data = pd.get_dummies(
    data= cust_order[[
        'customer_unique_id',
        'order_purchase_timestamp',
        'product_category_name_english'
    ]], 
    columns= ['product_category_name_english'],
    prefix= 'catg'
).groupby([
    'customer_unique_id',
    'order_purchase_timestamp'
]).sum()

txn_data = txn_data.reset_index()
txn_data

,customer_unique_id,order_purchase_timestamp,catg_agro_industry_and_commerce,catg_air_conditioning,catg_art,catg_arts_and_craftmanship,catg_audio,catg_auto,catg_baby,catg_bed_bath_table,...,catg_security_and_services,catg_signaling_and_security,catg_small_appliances,catg_small_appliances_home_oven_and_coffee,catg_sports_leisure,catg_stationery,catg_tablets_printing_image,catg_telephony,catg_toys,catg_watches_gifts
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98376,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06-08 21:00:36,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98377,fffea47cd6d3cc0a88bd621562a9d061,2017-12-10 20:07:56,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
98378,ffff371b4d645b6ecea244b27531430a,2017-02-07 15:49:16,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
98379,ffff5962728ec6157033ef9805bacc48,2018-05-02 15:17:41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [212]:
txn_data.to_csv('./Olist_data/transaction_data.csv', index= False)

### Payment data

The olist_order_payments_dataset contains information on the payments carried out by the customer. Every order_id has a payment linked to it. The payment can be done via multiple sources such as credit cards or vouchers etc and can be done in installments. If a payment for an order is broken down and paid via two or more different payment methods then that many rows with a sequence number is recorded.

In [47]:
payments = pd.read_csv('/home/raj/Downloads/Olist-Ecomm data/olist_order_payments_dataset.csv')
payments.head(5)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


About 75% of the payment methods are via Credit cards and the rest 25% divided between other forms. Due to this, the payment method is not an important attribute when analyzing payments. Hence neglecting payment methods and the sequence associated with it. The rows are grouped by order_id and the installments and payment value are summed. This reduces every order to one entry and giving the total payment value and total number of installments value was paid in irrespective of method.

In [48]:
order_payment = payments[[
    'order_id',
    'payment_installments',
    'payment_value'
]].groupby('order_id').sum().reset_index()

order_payment.head(5)

,order_id,payment_installments,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,2,72.19
1,00018f77f2f0320c557190d7a144bdd3,3,259.83
2,000229ec398224ef6ca0657da4fc703e,5,216.87
3,00024acbcdf0a6daa1e931b038114c75,2,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,3,218.04


This payment information is again joined with the customer order dataset. Now the customer order dataset gives information on payments made for each order as well.

In [49]:
cust_ord_payment = pd.merge(
    left= cust_order,
    right= order_payment,
    on= 'order_id',
    how= 'left'
)

cust_ord_payment.head(5)

,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,customer_unique_id,product_category_name_english,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,1,87285b34884572647811a353c7ac498a,29.99,8.72,7c396fd4830fd04220f754e42b4e5bff,housewares,3.0,38.71
1,128e10d95713541c87cd1a2e48201934,2017-08-15 18:29:31,1,87285b34884572647811a353c7ac498a,29.99,7.78,3a51803cc0d012c3b5dc8b7528cb05f7,housewares,3.0,37.77
2,0e7e841ddf8f8f2de2bad69267ecfbcf,2017-08-02 18:24:47,1,87285b34884572647811a353c7ac498a,29.99,7.78,ef0996a1a279c26e7ecbd737be23d235,housewares,1.0,37.77
3,bfc39df4f36c3693ff3b63fcbea9e90a,2017-10-23 23:26:46,1,87285b34884572647811a353c7ac498a,29.99,14.10,e781fdcc107d13d865fc7698711cc572,housewares,1.0,44.09
4,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,1,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76,af07308b275d755c9edb36a90c618231,perfumery,1.0,141.46


The above dataset doesnot give a complete idea as to what the customers have bought over time and how do they usually pay. Such habits are important when we want to understand customer behaviours. Thus dummy columns for every product category are created and summed for each customer (grouped by customer_unique_id).
One row here shows what the customer has bought over the 2 years summed up.

In [50]:
cust_txn = pd.get_dummies(
    data= cust_ord_payment[[
        'customer_unique_id',
        'product_category_name_english',
    ]], 
    columns= ['product_category_name_english'],
    prefix= "catg"
).groupby('customer_unique_id').sum().reset_index()

cust_txn.head(5)

,customer_unique_id,catg_agro_industry_and_commerce,catg_air_conditioning,catg_art,catg_arts_and_craftmanship,catg_audio,catg_auto,catg_baby,catg_bed_bath_table,catg_books_general_interest,...,catg_security_and_services,catg_signaling_and_security,catg_small_appliances,catg_small_appliances_home_oven_and_coffee,catg_sports_leisure,catg_stationery,catg_tablets_printing_image,catg_telephony,catg_toys,catg_watches_gifts
0,0000366f3b9a7992bf8c76cfdf3221e2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0000f46a3911fa3c0805444483337064,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0000f6ccb0745a6a4b88665a16c9f078,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0004aac84e0df4da2b147fca70cf8255,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Similarly for every customer an average of payment installments and payment value is calculated. This shows on average how much does the customer shop for and in how many installments does the customer pay the amount.

In [51]:
cust_mean = cust_ord_payment[[
    'customer_unique_id',
    'payment_installments',
    'payment_value'
]].groupby('customer_unique_id').mean().reset_index()

cust_mean.head(5)

,customer_unique_id,payment_installments,payment_value
0,0000366f3b9a7992bf8c76cfdf3221e2,8.0,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1.0,27.19
2,0000f46a3911fa3c0805444483337064,8.0,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,4.0,43.62
4,0004aac84e0df4da2b147fca70cf8255,6.0,196.89


The two datasets above are combined to show what customers have bought over the span of 2 years and on average how do they pay the amount.

In [52]:
cust_payment = pd.merge(
    left= cust_txn,
    right= cust_mean,
    on= 'customer_unique_id',
    how= 'inner'
)

cust_payment.head(5)

,customer_unique_id,catg_agro_industry_and_commerce,catg_air_conditioning,catg_art,catg_arts_and_craftmanship,catg_audio,catg_auto,catg_baby,catg_bed_bath_table,catg_books_general_interest,...,catg_small_appliances,catg_small_appliances_home_oven_and_coffee,catg_sports_leisure,catg_stationery,catg_tablets_printing_image,catg_telephony,catg_toys,catg_watches_gifts,payment_installments,payment_value
0,0000366f3b9a7992bf8c76cfdf3221e2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,8.0,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,27.19
2,0000f46a3911fa3c0805444483337064,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,8.0,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,4.0,43.62
4,0004aac84e0df4da2b147fca70cf8255,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,6.0,196.89


In [219]:
cust_payment.to_csv('./Olist_data/customer_payment.csv', index= False)

### Seller location data

The olist_sellers_dataset contains locations information the sellers in terms of zip code, city and state. The sellers list their products on the website and are then responsible for dispatching the delivery.

In [56]:
sellers = pd.read_csv('/home/raj/Downloads/Olist-Ecomm data/olist_sellers_dataset.csv')
sellers.head(5)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


Similar to customer, the sellers dataset is also combined with geolocation dataset to get latitude and longitude of the centroid of zipcode in which the seller resides.

In [57]:
seller_loc = pd.merge(
    left= sellers,
    right= loc_coordnts,
    left_on= 'seller_zip_code_prefix',
    right_on= 'geolocation_zip_code_prefix', 
    how= 'left'
)

seller_loc.head(5)

,seller_id,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,13023.0,-22.893848,-47.061337
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,13844.0,-22.383437,-46.947927
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,20031.0,-22.909572,-43.177703
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,4195.0,-23.657242,-46.612831
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,12914.0,-22.964803,-46.534419


The ord_items dataset is joined with orders dataset to get order_id, seller_id and customer_id mappings. This shows that for every temporary customer_id, which seller is assigned for the order and various milestones of delivery in timestamp.

In [58]:
cols= [
    'order_id',
    'customer_id',
    'order_purchase_timestamp',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
    'seller_id'
]

order_seller = pd.merge(
    left= ord_items,
    right= orders,
    on= 'order_id',
    how= 'left'
)[cols]

order_seller.head(5)

,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,seller_id
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-20 23:43:48,2017-09-29 00:00:00,48436dade18ac8b2bce089ec2a041202
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-05-12 16:04:24,2017-05-15 00:00:00,dd7ddc04e1b6c2c614352b383efe2d36
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-22 13:19:16,2018-02-05 00:00:00,5b51032eddd242adc84c38acab88f23d
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-14 13:32:39,2018-08-20 00:00:00,9d7a1d34a5052409006425275ba1c2b4
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-03-01 16:42:31,2017-03-17 00:00:00,df560393f3a51e74553ab94004ba5c87


This dataset is now joined with customer data to get the unique customer id mapped to seller and location of the customer.

In [59]:
cols= [
    'customer_id',
    'customer_unique_id',
    'geolocation_lat',
    'geolocation_lng',
    'customer_city'
]

cust_seller = pd.merge(
    left= order_seller,
    right= customer_data[cols],
    on= 'customer_id',
    how= 'left'
)

cust_seller.head(5)

,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,seller_id,customer_unique_id,geolocation_lat,geolocation_lng,customer_city
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-20 23:43:48,2017-09-29 00:00:00,48436dade18ac8b2bce089ec2a041202,871766c5855e863f6eccc05f988b23cb,-21.762775,-41.309633,campos dos goytacazes
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-05-12 16:04:24,2017-05-15 00:00:00,dd7ddc04e1b6c2c614352b383efe2d36,eb28e67c4c0b83846050ddfb8a35d051,-20.220527,-50.903424,santa fe do sul
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-22 13:19:16,2018-02-05 00:00:00,5b51032eddd242adc84c38acab88f23d,3818d81c6709e39d06b2738a8d3a2474,-19.870305,-44.593326,para de minas
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-14 13:32:39,2018-08-20 00:00:00,9d7a1d34a5052409006425275ba1c2b4,af861d436cfc08b2c2ddefd0ba074622,-23.089925,-46.611654,atibaia
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-03-01 16:42:31,2017-03-17 00:00:00,df560393f3a51e74553ab94004ba5c87,64b576fb70d441e8f1b2d7d446e483c5,-23.243402,-46.827614,varzea paulista


Finally the above dataset is merged with the seller information to get a mapping of customer locations to seller locations for every order.

In [60]:
cols= [
    'seller_id',
    'seller_city',
    'geolocation_lat',
    'geolocation_lng'
]

delivery = pd.merge(
    left= cust_seller,
    right= seller_loc[cols],
    on= 'seller_id',
    how= 'left'
)

delivery.head(5)

,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,seller_id,customer_unique_id,geolocation_lat_x,geolocation_lng_x,customer_city,seller_city,geolocation_lat_y,geolocation_lng_y
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-20 23:43:48,2017-09-29 00:00:00,48436dade18ac8b2bce089ec2a041202,871766c5855e863f6eccc05f988b23cb,-21.762775,-41.309633,campos dos goytacazes,volta redonda,-22.496953,-44.127492
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-05-12 16:04:24,2017-05-15 00:00:00,dd7ddc04e1b6c2c614352b383efe2d36,eb28e67c4c0b83846050ddfb8a35d051,-20.220527,-50.903424,santa fe do sul,sao paulo,-23.565096,-46.518565
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-22 13:19:16,2018-02-05 00:00:00,5b51032eddd242adc84c38acab88f23d,3818d81c6709e39d06b2738a8d3a2474,-19.870305,-44.593326,para de minas,borda da mata,-22.262584,-46.171124
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-14 13:32:39,2018-08-20 00:00:00,9d7a1d34a5052409006425275ba1c2b4,af861d436cfc08b2c2ddefd0ba074622,-23.089925,-46.611654,atibaia,franca,-20.553624,-47.387359
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-03-01 16:42:31,2017-03-17 00:00:00,df560393f3a51e74553ab94004ba5c87,64b576fb70d441e8f1b2d7d446e483c5,-23.243402,-46.827614,varzea paulista,loanda,-22.929384,-53.135873


All irrelevant columns are removed. The resultant dataset now contains information on customer's purchase dates, delivery dates and seller assigned.

In [61]:
delivery.drop(columns=[
    'order_id',
    'customer_id'
], inplace= True)

delivery.rename(columns={
    'geolocation_lat_x':'customer_lat',
    'geolocation_lng_x':'customer_lng',
    'geolocation_lat_y':'seller_lat',
    'geolocation_lng_y':'seller_lng'
}, inplace= True)

delivery.head(5)

,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,seller_id,customer_unique_id,customer_lat,customer_lng,customer_city,seller_city,seller_lat,seller_lng
0,2017-09-13 08:59:02,2017-09-20 23:43:48,2017-09-29 00:00:00,48436dade18ac8b2bce089ec2a041202,871766c5855e863f6eccc05f988b23cb,-21.762775,-41.309633,campos dos goytacazes,volta redonda,-22.496953,-44.127492
1,2017-04-26 10:53:06,2017-05-12 16:04:24,2017-05-15 00:00:00,dd7ddc04e1b6c2c614352b383efe2d36,eb28e67c4c0b83846050ddfb8a35d051,-20.220527,-50.903424,santa fe do sul,sao paulo,-23.565096,-46.518565
2,2018-01-14 14:33:31,2018-01-22 13:19:16,2018-02-05 00:00:00,5b51032eddd242adc84c38acab88f23d,3818d81c6709e39d06b2738a8d3a2474,-19.870305,-44.593326,para de minas,borda da mata,-22.262584,-46.171124
3,2018-08-08 10:00:35,2018-08-14 13:32:39,2018-08-20 00:00:00,9d7a1d34a5052409006425275ba1c2b4,af861d436cfc08b2c2ddefd0ba074622,-23.089925,-46.611654,atibaia,franca,-20.553624,-47.387359
4,2017-02-04 13:57:51,2017-03-01 16:42:31,2017-03-17 00:00:00,df560393f3a51e74553ab94004ba5c87,64b576fb70d441e8f1b2d7d446e483c5,-23.243402,-46.827614,varzea paulista,loanda,-22.929384,-53.135873


In [62]:
delivery.to_csv('/home/raj/Github/Olist-business-analysis/Joined data/delivery_data.csv', index= False)

### Order reviews

The olist_order_reviews dataset contains the reviews and stars recieved for products based on order_id. The reviews are in Portuguese and need to be translated to English. Furthermore it contains information on when the review was posted and when was it replied to by concerned authorities.

In [245]:
reviews = pd.read_csv('/home/raj/Downloads/Olist-Ecomm data/olist_order_reviews_dataset.csv')
reviews.head(5)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


The reviews here are given for each order. The aim is to collect the reviews product wise so as to perform analysis on how each product bought is percieved by the customers.<br>

For each product the orders are found and thus the corresponding reviews. These are grouped together and mapped to product_id. This was for each product_id we have a set of reviews retieved from orders containing those products.

In [340]:
cols = list(reviews.columns)
cols.append('product_id')
prod_review = pd.DataFrame(columns= cols)

def product_review(prod):
    
    global prod_review
    oid = ord_items[ord_items.product_id == prod].order_id
    rid = reviews[reviews.order_id.isin(oid)].review_id
    data = reviews[reviews.review_id.isin(rid)].copy()
    data['product_id'] = prod
    prod_review = pd.concat([prod_review,data], ignore_index=True)
    
products.product_id.apply(product_review)

0        None
1        None
2        None
3        None
4        None
         ... 
32946    None
32947    None
32948    None
32949    None
32950    None
Name: product_id, Length: 32951, dtype: object

In [353]:
prod_review.head(5)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,product_id
0,d71da8fd8c6e3adef26be965f065b8a1,e17e4f88e31525f7deef66779844ddce,5,Vale a pena,Super adooorei o delineador ele é bem preto e ...,2018-04-28 00:00:00,2018-05-04 00:55:02,1e9e8ef04dbcff4541ed26657ea517e5
1,19866247db89845ef558ef19a6c26fcd,5236307716393b7114b53ee991f36956,5,NaN,Excelente apontador! Em 3 segundos uma ponta p...,2018-02-17 00:00:00,2018-02-18 23:24:14,3aa071139cb16b67ca9e5dea641aaa2f
2,32921b4e09c9a19d61fe46ce04231b84,01f66e58769f84129811d43eefd187fb,5,NaN,NaN,2018-07-14 00:00:00,2018-07-16 16:20:04,96bd76ec8810374ed1b65e291975717f
3,70949d129965fb1ee32aaccec720a19e,143d00a4f2dde4e0364ee1821577adb3,1,produto divergente,modelo comprado: travesseiro memoria carinho B...,2018-08-07 00:00:00,2018-08-08 11:53:04,cef67bcfe19066a932b7673e239eb23d
4,74c9c7d0bc153bbbc92e4c1597b8ba55,86cafb8794cb99a9b1b77fc8e48fbbbb,5,NaN,NaN,2018-04-17 00:00:00,2018-04-18 04:29:47,9dc1a7de274444849c219cff195d0b71


In [354]:
prod_review.to_csv('./Olist_data/porduct_review.csv', index= False)